# Verify the Correctness of Exported Model and Compare the Performance

In this tutorial, we are going to show:
- how to verify the correctness of the exported model
- how to compare the performance with the original model

We choose PyTorch to export the ONNX model, and use Caffe2 as the backend.
After that, the outputs and performance of two models are compared.

To run this tutorial, please make sure that `caffe2`, `pytorch`, `onnx`, `onnx-caffe2` are already installed.

First, let's create a PyTorch model and prepare the inputs of the model.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import io
import numpy as np
import torch
import onnx
import torch.nn as nn
import torch.nn.functional as F

from caffe2.proto import caffe2_pb2
from caffe2.python import core
from torch.autograd import Variable
from onnx_caffe2.backend import Caffe2Backend
from onnx_caffe2.helper import c2_native_run_net, name_inputs, save_caffe2_net, load_caffe2_net, \
    benchmark_caffe2_model, benchmark_pytorch_model


class MNIST(nn.Module):

    def __init__(self):
        super(MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)


# Create a pytorch model.
pytorch_model = MNIST()
pytorch_model.train(False)

# Make the inputs in tuple format.
inputs = (Variable(torch.randn(3, 1, 28, 28), requires_grad=True), )

Run the PyTorch exporter to generate an ONNX model.

In [2]:
# Export an ONNX model.
f = io.BytesIO()
torch.onnx.export(pytorch_model, inputs, f, verbose=True)
onnx_model = onnx.ModelProto.FromString(f.getvalue())

# Check whether the onnx_model is valid or not.
print("Check the ONNX model.")
onnx.checker.check_model(onnx_model)

graph(%0 : Float(3, 1, 28, 28)
      %1 : Float(10, 1, 5, 5)
      %2 : Float(10)
      %3 : Float(20, 10, 5, 5)
      %4 : Float(20)
      %5 : Float(50, 320)
      %6 : Float(50)
      %7 : Float(10, 50)
      %8 : Float(10)) {
  %9 : UNKNOWN_TYPE = Conv[kernel_shape=[5, 5], strides=[1, 1], pads=[0, 0, 0, 0], dilations=[1, 1], group=1](%0, %1)
  %10 : Float(3, 10, 24, 24) = Add[broadcast=1, axis=1](%9, %2)
  %11 : Float(3, 10, 12, 12) = MaxPool[kernel_shape=[2, 2], pads=[0, 0], strides=[2, 2]](%10)
  %12 : Float(3, 10, 12, 12) = Relu(%11)
  %13 : UNKNOWN_TYPE = Conv[kernel_shape=[5, 5], strides=[1, 1], pads=[0, 0, 0, 0], dilations=[1, 1], group=1](%12, %3)
  %14 : Float(3, 20, 8, 8) = Add[broadcast=1, axis=1](%13, %4)
  %15 : Float(3, 20, 4, 4) = MaxPool[kernel_shape=[2, 2], pads=[0, 0], strides=[2, 2]](%14)
  %16 : Float(3, 20, 4, 4) = Relu(%15)
  %17 : Float(3, 320) = Reshape[shape=[-1, 320]](%16)
  %18 : Float(320!, 50!) = Transpose[perm=[1, 0]](%5)
  %20 : Float(3, 50) = Gemm[alp

Now, we have an ONNX model, let's turn it into a Caffe2 one.

In [3]:
# Convert the ONNX model to a Caffe2 model.
print("Convert the model to a Caffe2 model.")
init_net, predict_net = Caffe2Backend.onnx_graph_to_caffe2_net(onnx_model.graph, device="CPU")

Convert the model to a Caffe2 model.


Caffe2 takes a list of numpy array as inputs. So we need to change the format.

In [4]:
# Prepare the inputs for Caffe2.
caffe2_inputs = [var.data.numpy() for var in inputs]

The following code shows how to save and load a Caffe2 model. It is purely for demonstration purpose here.

In [5]:
# Save the converted Caffe2 model in the protobuf files. (Optional)
init_file = "./output/mymodel_init.pb"
predict_file = "./output/mymodel_predict.pb"
save_caffe2_net(init_net, init_file, output_txt=False)
save_caffe2_net(predict_net, predict_file, output_txt=True)

# Load the Caffe2 model.
init_net = load_caffe2_net(init_file)
predict_net = load_caffe2_net(predict_file)

Run PyTorch and Caffe2 models separately, and get the results.

In [6]:
# Compute the results using the PyTorch model.
pytorch_results = pytorch_model(*inputs)

# Compute the results using the Caffe2 model.
_, caffe2_results = c2_native_run_net(init_net, predict_net, name_inputs(onnx_model, caffe2_inputs))

Now we have the results, let's check the correctness of the exported model.
If no assertion fails, our model has achieved expected precision.

In [7]:
# Check the decimal precision of the exported Caffe2.
expected_decimal = 5
for p, c in zip([pytorch_results], caffe2_results):
    np.testing.assert_almost_equal(p.data.cpu().numpy(), c, decimal=expected_decimal)
print("The exported model achieves {}-decimal precision.".format(expected_decimal))

The exported model achieves 5-decimal precision.


The following code measures the performance of PyTorch and Caffe2 models.
We report:
- Execution time per iteration
- Iterations per second

In [8]:
pytorch_time = benchmark_pytorch_model(pytorch_model, inputs)
caffe2_time = benchmark_caffe2_model(init_net, predict_net)

print("PyTorch model's execution time is {} milliseconds/ iteration, {} iterations per second.".format(
    pytorch_time, 1000 / pytorch_time))
print("Caffe2 model's execution time is {} milliseconds / iteration, {} iterations per second".format(
    caffe2_time, 1000 / caffe2_time))

PyTorch model's execution time is 0.637650489807 milliseconds/ iteration, 1568.25724434 iterations per second.
Caffe2 model's execution time is 0.43121188879 milliseconds / iteration, 2319.04552262 iterations per second
